# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pickle
from scipy import stats
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
import numpy as np
import pandas as pd
import warnings
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
from pypdf import PdfReader
from pdfminer.high_level import extract_text

# Load data

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

# Результаты анализов

In [ ]:
fns = [
    '1013265046БелоконьЮлияВикторовна',
    '9077400106КубановаДианаВалентиновна',
    '9077400276ЛюбичЕвгенияВалерьевна',
    '9077400403НасоноваОльгаСергеевна',
    '9077400498ЖамалитдиноваОльгаХайдаровна',
    '9077400596ИвановаМаринаАлександровна',
    '9077401026ИзбенкоВикторияСергеевна',
    'дек 5.21'
]

feats = pd.read_excel(f"{path}/Результаты анализов/features.xlsx", index_col=0)
feats_same_line = feats.loc[feats['line'] == 0, :]
feats_diff_line = feats.loc[feats['line'] != 0, :]

feats_same_line_dict = dict(zip(feats_same_line.index.values, feats_same_line['feature'].values))
feats_diff_line_dict = dict(zip(feats_diff_line.index.values, feats_diff_line['feature'].values))

df = pd.DataFrame(columns=['Pages', 'File', '№ направления', 'ФИО', 'Дата рождения', 'Дата обследования', 'Пол'] + list(feats['feature']))

missed_pages = {}
missed_lines = []

with open(f"{path}/Результаты анализов/skip_starts.txt") as f:
    skip_starts = tuple(list(set(f.read().splitlines())))

for fn in fns:
    print(fn)
    
    missed_pages[fn] = []
    
    reader = PdfReader(f"{path}/Результаты анализов/{fn}.pdf")
    
    for page_id, page in tqdm(enumerate(reader.pages)):
        lines = page.extract_text().splitlines()
        
        if lines[0] == 'Фамилия:':
        
            sample_id = lines[3]
            
            if sample_id in df.index:
                df.at[sample_id, 'Pages'] += f' {page_id}'
            else:
                df.at[sample_id, 'Pages'] = f'{page_id}'

            df.at[sample_id, 'File'] = fn
            df.at[sample_id, '№ направления'] = sample_id
            df.at[sample_id, 'ФИО'] = lines[12].capitalize() + ' ' + re.findall(r"(.*)Имя", lines[11])[0]
            df.at[sample_id, 'Дата рождения'] = lines[4]
            df.at[sample_id, 'Дата обследования'] = re.findall(r"Дата: (.*)", lines[10])[0]
            df.at[sample_id, 'Пол'] = re.findall(r"Пол: (.+)", lines[7])[0][0]

            for line_id, line in enumerate(lines):
                # line = line.replace("не обнаружено", "0.0")
                if line in feats_diff_line_dict:
                    target_line = lines[line_id + feats_diff_line.at[line, 'line']]
                    line_parse = re.findall(fr"([-+]?(?:\d+\.\d+|\d+|\.\d+)).*", target_line)
                    df.at[sample_id, feats_diff_line_dict[line]] = line_parse[0]
                else:
                    line = line.replace(" - ", "-")
                    line_parse = re.findall(fr"(.*\S)\s+([-+]?(?:\d+\.\d+|\d+|\.\d+)) ", line)
                    if len(line_parse) > 0:
                        if line_parse[0][0] in feats_same_line_dict:
                            df.at[sample_id, feats_same_line_dict[line_parse[0][0]]] = line_parse[0][1]
                        else:
                            if not line.startswith(skip_starts):
                                missed_lines.append(line)
        else:
            missed_pages[fn].append(page_id)

df = df.apply(pd.to_numeric, errors='ignore')
df.to_excel(f"{path}/Результаты анализов/data.xlsx")

In [ ]:
print('\n'.join(missed_lines))

In [ ]:
from collections import Counter
words = ' '.join(missed_lines).split(' ')
word_and_freq = dict(Counter(words).most_common())
df_word_and_freq = pd.Series(word_and_freq, name='Count').to_frame()
df_word_and_freq.to_excel(f"{path}/Результаты анализов/words.xlsx")

In [ ]:
print(missed_pages)

In [ ]:
feats = pd.read_excel(f"{path}/Результаты анализов/features.xlsx", index_col='feature')
feats_old = pd.read_excel(f"{path}/first_examples/feats.xlsx", index_col='feature')
feats_remains = feats_old.index.difference(feats.index).to_list()
print('\n'.join(feats_remains))

In [ ]:
fn = '1013265046БелоконьЮлияВикторовна'

reader = PdfReader(f"{path}/Результаты анализов/{fn}.pdf")
print(reader.pages[4].extract_text())

# Bioelectrical impedance analysis

In [ ]:
feats_rows_bia = {
    'ID': 3,
    'Дата и время обследования': 4,
    'Возраст': 5,
    'Пол': 5,
    'Рост': 6,
    'Вес': 6,
    'Окр. талии, см': 7,
    'Окр. бедер, см': 7,
    'Сопрот. акт. на 5 кГц, Ом': 12,
    'Сопрот. акт. на 50 кГц, Ом': 12,
    'Сопрот. реакт. на 50 кГц, Ом': 12,
    'Фазовый угол (50 кГц), град.': 13,
    'Клеточная жидкость, кг': 14,
    'Удельный основной обмен, ккал/м²/сут.': 15,
    'Индекс массы тела': 18,
    'Индекс массы тела (% от середины нормы)': 21,
    'Жировая масса, кг': 23,
    'Жировая масса, кг (% от середины нормы)': 26,
    'Тощая масса, кг': 28,
    'Тощая масса, кг (% от середины нормы)': 31,
    'Активная клеточная масса, кг': 34,
    'Активная клеточная масса, кг (% от середины нормы)': 37,
    'Доля активной клеточной массы, %': 40,
    'Доля активной клеточной массы, % (% от середины нормы)': 43,
    'Скелетно-мышечная масса, кг': 46,
    'Скелетно-мышечная масса, кг (% от середины нормы)': 49,
    'Доля скелетно-мышечной массы, %': 53,
    'Доля скелетно-мышечной массы, % (% от середины нормы)': 56,
    'Основной обмен, ккал/сут.': 59,
    'Основной обмен, ккал/сут. (% от середины нормы)': 62,
    'Общая жидкость, кг': 64,
    'Общая жидкость, кг (% от середины нормы)': 67,
    'Внеклеточная жидкость, кг': 70,
    'Внеклеточная жидкость, кг (% от середины нормы)': 73,
    'Индекс талия-бедра': 75,
    'Индекс талия-бедра (% от середины нормы)': 78,
    'Доля жировой массы, %': 81,
    'Доля жировой массы, % (% от середины нормы)': 84,
    
    'Отношение внеклеточной и клеточной жидкостей': 33,
    
    'Индекс жировой массы, кг/м²': 20,
    'Индекс жировой массы, кг/м² (% от середины нормы)': 23,
    'Индекс тощей массы, кг/м²': 24,
    'Индекс тощей массы, кг/м² (% от середины нормы)': 27,
    'Индекс активной клеточной массы, кг/м²': 28,
    'Индекс активной клеточной массы, кг/м² (% от середины нормы)': 31,
    'Индекс скелетно-мышечной клеточной массы, кг/м²': 32,
    'Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)': 35,
    'Минеральная масса тела, кг': 38,
    'Минеральная масса тела, кг (% от середины нормы)': 41,
    'Минеральная масса мягких тканей, кг': 44,
    'Минеральная масса мягких тканей, кг (% от середины нормы)': 47,
    'Минеральная масса костной ткани, кг': 50,
    'Минеральная масса костной ткани, кг (% от середины нормы)': 53,
    'Доля минеральной массы в ТМ, %': 56,
    'Доля минеральной массы в ТМ, % (% от середины нормы)': 59,
    'Доля минеральной массы мягких тканей в ТМ, %': 63,
    'Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)': 66,
    'Доля минеральной массы костной ткани в ТМ, %': 70,
    'Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)': 73,
    'Шкала оценки риска метаболического синдрома по проценту жировой массы': 78,
    'Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)': 81
}

fns = [
    'Бурминская Светлана Александровна, #2, 2024-10-11 10_55_46',
    'Нестерова Наталья Григорьевна, #1, 2024-10-03 11_08_44',
    'Гуржий Юлия Андреевна, #1, 2024-07-05 10_31_22',
    'Задоя  Ирина Петровна, #1, 2024-09-26 08_12_36',
    'Мустафина Диана Рашидовна, #1, 2025-02-21 10_53_39',
    'Вотина Полина Игоревна, #1, 2024-08-14 08_43_21'
]

df_bia = pd.DataFrame(columns=list(feats_rows_bia.keys()))

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")
    
    # Page 1
    page = reader.pages[0]
    lines = page.extract_text().splitlines()
    sample_id = lines[feats_rows_bia['ID']]
    df_bia.at[sample_id, 'ID'] = sample_id
    date_time = re.findall(r"([0-9]{1,2}\.[0-9]{1,2}\.[0-9]{4})\s(\d{1,2}:\d{2}:\d{2})Дата\sобследования", lines[feats_rows_bia['Дата и время обследования']])[0]
    df_bia.at[sample_id, 'Дата и время обследования'] = f"{date_time[0]} {date_time[1]}"
    df_bia.at[sample_id, 'Возраст'] = float(re.findall(r"(\d+), (.)Возраст, лет \/ Пол", lines[feats_rows_bia['Возраст']])[0][0])
    df_bia.at[sample_id, 'Пол'] = re.findall(r"(\d+), (.)Возраст, лет \/ Пол", lines[feats_rows_bia['Пол']])[0][1]
    df_bia.at[sample_id, 'Рост'] = float(re.findall(r"(\d+) \/ (\d+\.*\d*)Рост, см \/ Вес, кг", lines[feats_rows_bia['Рост']])[0][0])
    df_bia.at[sample_id, 'Вес'] = float(re.findall(r"(\d+) \/ (\d+\.*\d*)Рост, см \/ Вес, кг", lines[feats_rows_bia['Вес']])[0][1])
    df_bia.at[sample_id, 'Окр. талии, см'] = float(re.findall(r"(\d+) \/ (\d+)Окр. талии \/ Окр. бедер, см", lines[feats_rows_bia['Окр. талии, см']])[0][0])
    df_bia.at[sample_id, 'Окр. бедер, см'] = float(re.findall(r"(\d+) \/ (\d+)Окр. талии \/ Окр. бедер, см", lines[feats_rows_bia['Окр. бедер, см']])[0][1])
    df_bia.at[sample_id, 'Сопрот. акт. на 5 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows_bia['Сопрот. акт. на 5 кГц, Ом']])[0][0])
    df_bia.at[sample_id, 'Сопрот. акт. на 50 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows_bia['Сопрот. акт. на 50 кГц, Ом']])[0][1])
    df_bia.at[sample_id, 'Сопрот. реакт. на 50 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows_bia['Сопрот. реакт. на 50 кГц, Ом']])[0][2])
    df_bia.at[sample_id, 'Фазовый угол (50 кГц), град.'] = float(lines[feats_rows_bia['Фазовый угол (50 кГц), град.']])
    df_bia.at[sample_id, 'Клеточная жидкость, кг'] = float(lines[feats_rows_bia['Клеточная жидкость, кг']])
    df_bia.at[sample_id, 'Удельный основной обмен, ккал/м²/сут.'] = float(lines[feats_rows_bia['Удельный основной обмен, ккал/м²/сут.']])
    df_bia.at[sample_id, 'Индекс массы тела'] = float(lines[feats_rows_bia['Индекс массы тела']])
    df_bia.at[sample_id, 'Индекс массы тела (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс массы тела (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Жировая масса, кг'] = float(lines[feats_rows_bia['Жировая масса, кг']])
    df_bia.at[sample_id, 'Жировая масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Жировая масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Тощая масса, кг'] = float(lines[feats_rows_bia['Тощая масса, кг']])
    df_bia.at[sample_id, 'Тощая масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Тощая масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Активная клеточная масса, кг'] = float(lines[feats_rows_bia['Активная клеточная масса, кг']])
    df_bia.at[sample_id, 'Активная клеточная масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Активная клеточная масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля активной клеточной массы, %'] = float(lines[feats_rows_bia['Доля активной клеточной массы, %']])
    df_bia.at[sample_id, 'Доля активной клеточной массы, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля активной клеточной массы, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Скелетно-мышечная масса, кг'] = float(lines[feats_rows_bia['Скелетно-мышечная масса, кг']])
    df_bia.at[sample_id, 'Скелетно-мышечная масса, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Скелетно-мышечная масса, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля скелетно-мышечной массы, %'] = float(lines[feats_rows_bia['Доля скелетно-мышечной массы, %']])
    df_bia.at[sample_id, 'Доля скелетно-мышечной массы, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля скелетно-мышечной массы, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Основной обмен, ккал/сут.'] = float(lines[feats_rows_bia['Основной обмен, ккал/сут.']])
    df_bia.at[sample_id, 'Основной обмен, ккал/сут. (% от середины нормы)'] = float(lines[feats_rows_bia['Основной обмен, ккал/сут. (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Общая жидкость, кг'] = float(lines[feats_rows_bia['Общая жидкость, кг']])
    df_bia.at[sample_id, 'Общая жидкость, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Общая жидкость, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Внеклеточная жидкость, кг'] = float(lines[feats_rows_bia['Внеклеточная жидкость, кг']])
    df_bia.at[sample_id, 'Внеклеточная жидкость, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Внеклеточная жидкость, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс талия-бедра'] = float(lines[feats_rows_bia['Индекс талия-бедра']])
    df_bia.at[sample_id, 'Индекс талия-бедра (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс талия-бедра (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля жировой массы, %'] = float(lines[feats_rows_bia['Доля жировой массы, %']])
    df_bia.at[sample_id, 'Доля жировой массы, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля жировой массы, % (% от середины нормы)']].replace('%', ''))

    # Page 3
    lines = reader.pages[2].extract_text().splitlines()
    df_bia.at[sample_id, 'Отношение внеклеточной и клеточной жидкостей'] = float(re.findall(r"Ваш показатель ВКЖ\/КЖ составляет: (\d+\.*\d*) \(диапазон нормальных значений", lines[feats_rows_bia['Отношение внеклеточной и клеточной жидкостей']])[0])

    # Page 5
    lines = reader.pages[4].extract_text().splitlines()
    df_bia.at[sample_id, 'Индекс жировой массы, кг/м²'] = float(lines[feats_rows_bia['Индекс жировой массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс жировой массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс жировой массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс тощей массы, кг/м²'] = float(lines[feats_rows_bia['Индекс тощей массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс тощей массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс тощей массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс активной клеточной массы, кг/м²'] = float(lines[feats_rows_bia['Индекс активной клеточной массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс активной клеточной массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс активной клеточной массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Индекс скелетно-мышечной клеточной массы, кг/м²'] = float(lines[feats_rows_bia['Индекс скелетно-мышечной клеточной массы, кг/м²']])
    df_bia.at[sample_id, 'Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows_bia['Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Минеральная масса тела, кг'] = float(lines[feats_rows_bia['Минеральная масса тела, кг']])
    df_bia.at[sample_id, 'Минеральная масса тела, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Минеральная масса тела, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Минеральная масса мягких тканей, кг'] = float(lines[feats_rows_bia['Минеральная масса мягких тканей, кг']])
    df_bia.at[sample_id, 'Минеральная масса мягких тканей, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Минеральная масса мягких тканей, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Минеральная масса костной ткани, кг'] = float(lines[feats_rows_bia['Минеральная масса костной ткани, кг']])
    df_bia.at[sample_id, 'Минеральная масса костной ткани, кг (% от середины нормы)'] = float(lines[feats_rows_bia['Минеральная масса костной ткани, кг (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля минеральной массы в ТМ, %'] = float(lines[feats_rows_bia['Доля минеральной массы в ТМ, %']])
    df_bia.at[sample_id, 'Доля минеральной массы в ТМ, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля минеральной массы в ТМ, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля минеральной массы мягких тканей в ТМ, %'] = float(lines[feats_rows_bia['Доля минеральной массы мягких тканей в ТМ, %']])
    df_bia.at[sample_id, 'Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Доля минеральной массы костной ткани в ТМ, %'] = float(lines[feats_rows_bia['Доля минеральной массы костной ткани в ТМ, %']])
    df_bia.at[sample_id, 'Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)'] = float(lines[feats_rows_bia['Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)']].replace('%', ''))
    df_bia.at[sample_id, 'Шкала оценки риска метаболического синдрома по проценту жировой массы'] = float(lines[feats_rows_bia['Шкала оценки риска метаболического синдрома по проценту жировой массы']])
    df_bia.at[sample_id, 'Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)'] = float(lines[feats_rows_bia['Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)']].replace('%', ''))

df_bia = df_bia.apply(pd.to_numeric, errors='ignore')

# Different analyses 1

In [ ]:
feats_anls = {
    1: {
        'Дата и время обследования': 16,
        'Возраст': 17,
        'Пол': 7,
        'Аланинаминотрансфераза (АЛТ), Ед/л': 25,
        'Аспартатаминотрансфераза (АСТ), Ед/л': 26,
        'Витамин В9 (фолиевая кислота), нг/мл': 27,
        'Витамин В12 (цианкобаламин), пг/мл': 28,
        'Общий белок, г/л': 29,
        'Билирубин непрямой, мкмоль/л': 30,
        'Билирубин общий, мкмоль/л': 31,
        'Билирубин прямой, мкмоль/л': 32,
        'Мочевая кислота, мкмоль/л': 33,
        'Креатинин, мкмоль/л': 34,
        'Глюкоза, ммоль/л': 35,
        'Триглицериды, ммоль/л': 44,
        'ЛПОНП, ммоль/л': 50,
        'Коэффициент атерогенности': 51
    },
    2: {
        'Холестерин общий, ммоль/л': 23,
        'Холестерин липопротеидов высокой плотности (ЛПВП, HDL), ммоль/л': 37,
        'Холестерин не-ЛПВП, ммоль/л': 44,
        'Холестерин липопротеидов низкой плотности (ЛПНП, LDL), ммоль/л': 57,
        'Гомоцистеин, мкмоль/л': 65,
    },
    3: {
        'Медь, мкмоль/л': 23,
        'Магний, ммоль/л': 24,
        'Цинк, мкмоль/л': 25,
        'Железо, мкмоль/л': 26,
        'Ферритин, мкг/л': 27,
    },
    4: {
        'Тиреотропный гормон (ТТГ), мкМЕ/мл': 25,
        'Тироксин свободный (Т4 свободный), пмоль/л': 31,
        'Трийодтиронин свободный (Т3 свободный), пмоль/л': 34,
        'Антитела к тиреоглобулину (Анти-ТГ), МЕ/мл': 35,
        'Антитела к микросомальной тиреопероксидазе (Анти-ТПО), МЕ/мл': 38,
        'Фолликулостимулирующий гормон (ФСГ), мМЕ/мл': 39,
    },
    5: {
        'Лютеинизирующий гормон (ЛГ), мМЕ/мл': 23,
        'Пролактин, мМЕ/л': 40,
        'Прогестерон, нг/мл': 46,
    },
    6: {
        'Эстрадиол (Е2), пмоль/л': 23,
        'Тестостерон свободный, нмоль/л': 43,
        'Индекс свободных андрогенов, %': 44,
        'Тестостерон общий, нмоль/л': 45,
        'Глобулин, связывающий половые гормоны (ГСПГ, SHBG), нмоль/л': 48,
        'Инсулин, мкМЕ/мл': 49,
    },
    7: {
        'Кальций ионизированный, ммоль/л': 25,
    },
    8: {
        '25-OH витамин D, ИХЛА, суммарный (кальциферол), нг/мл': 27,
    },
    9: {
        'Эритроциты, 10*12/л': 25,
        'Гемоглобин, г/л': 26,
        'Гематокрит, %': 27,
        'Средний объем эритроцитов (MCV), фл': 28,
        'Среднее содержание гемоглобина в эритроците (МСН), пг/кл': 31,
        'Средняя концентрация Hb в эритроцитах (МСНС), г/дл': 34,
        'Отн.ширина распред.эритр.по объему (ст.отклонение), фл': 37,
        'Отн.ширина распред.эритр.по объему(коэфф.вариации), %': 40,
        'Тромбоциты, 10*9/л': 41,
        'Средний объем тромбоцитов (MPV), фл': 42,
        'Тромбокрит (PCT), %': 43,
        'Относит.ширина распред.тромбоцитов по объему (PDW), %': 46,
        'Лейкоциты, 10*9/л': 47,
        'Нейтрофилы, 10*9/л': 48,
        'Нейтрофилы, %': 49,
        'Эозинофилы, 10*9/л': 50,
        'Эозинофилы, %': 51,
        'Базофилы, 10*9/л': 52,
        'Базофилы, %': 53,
        'Моноциты, 10*9/л': 54,
        'Моноциты, %': 55,
        'Лимфоциты, 10*9/л': 56,
        'Лимфоциты, %': 57,
    },
    10: {
        'Гликированный гемоглобин А1с, %': 25,
    }
}

cols_anls = ['ID'] + list(itertools.chain.from_iterable([list(feats_anls[x].keys()) for x in feats_anls])) 
df_anls = pd.DataFrame(columns=list(cols_anls))

fns = [
    '9198800454ГуржийЮлияАндреевна',
    '9198800755ВотинаПолинаИгоревна',
    '9198801244НестероваНатальяГригорьевна',
    '9198802511МустафинаДианаРашидовна',
]

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")

    for page in reader.pages:
        lines = page.extract_text().splitlines()
        sample_id = lines[11].capitalize() + ' ' + re.findall(r"(.*)Имя", lines[10])[0]
        if lines[25].startswith('Аланинаминотрансфераза (АЛТ)'):
            page_id = 1
            df_anls.at[sample_id, 'ID'] = sample_id
            df_anls.at[sample_id, 'Дата и время обследования'] = lines[feats_anls[page_id]['Дата и время обследования']]
            df_anls.at[sample_id, 'Возраст'] = float(re.findall(r"\((\d+) .\)", lines[feats_anls[page_id]['Возраст']])[0])
            df_anls.at[sample_id, 'Пол'] = re.findall(r"Пол: (.+)", lines[feats_anls[page_id]['Пол']])[0][0]
            df_anls.at[sample_id, 'Аланинаминотрансфераза (АЛТ), Ед/л'] = re.findall(r"Аланинаминотрансфераза \(АЛТ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Аланинаминотрансфераза (АЛТ), Ед/л']])[0]
            df_anls.at[sample_id, 'Аспартатаминотрансфераза (АСТ), Ед/л'] = re.findall(r"Аспартатаминотрансфераза \(АСТ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Аспартатаминотрансфераза (АСТ), Ед/л']])[0]
            df_anls.at[sample_id, 'Витамин В9 (фолиевая кислота), нг/мл'] = re.findall(r"Витамин В9 \(фолиевая кислота\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Витамин В9 (фолиевая кислота), нг/мл']])[0]
            df_anls.at[sample_id, 'Витамин В12 (цианкобаламин), пг/мл'] = re.findall(r"Витамин В12 \(цианкобаламин\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Витамин В12 (цианкобаламин), пг/мл']])[0]
            df_anls.at[sample_id, 'Общий белок, г/л'] = re.findall(r"Общий белок (\d+\.*\d*) .*", lines[feats_anls[page_id]['Общий белок, г/л']])[0]
            df_anls.at[sample_id, 'Билирубин непрямой, мкмоль/л'] = re.findall(r"Билирубин непрямой (\d+\.*\d*) .*", lines[feats_anls[page_id]['Билирубин непрямой, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Билирубин общий, мкмоль/л'] = re.findall(r"Билирубин общий (\d+\.*\d*) .*", lines[feats_anls[page_id]['Билирубин общий, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Билирубин прямой, мкмоль/л'] = re.findall(r"Билирубин прямой (\d+\.*\d*) .*", lines[feats_anls[page_id]['Билирубин прямой, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Мочевая кислота, мкмоль/л'] = re.findall(r"Мочевая кислота (\d+\.*\d*) .*", lines[feats_anls[page_id]['Мочевая кислота, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Креатинин, мкмоль/л'] = re.findall(r"Креатинин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Креатинин, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Глюкоза, ммоль/л'] = re.findall(r"Глюкоза (\d+\.*\d*) .*", lines[feats_anls[page_id]['Глюкоза, ммоль/л']])[0]
            df_anls.at[sample_id, 'Триглицериды, ммоль/л'] = re.findall(r"Триглицериды (\d+\.*\d*) .*", lines[feats_anls[page_id]['Триглицериды, ммоль/л']])[0]
            df_anls.at[sample_id, 'ЛПОНП, ммоль/л'] = re.findall(r"ЛПОНП (\d+\.*\d*) .*", lines[feats_anls[page_id]['ЛПОНП, ммоль/л']])[0]
            df_anls.at[sample_id, 'Коэффициент атерогенности'] = re.findall(r"Коэффициент атерогенности (\d+\.*\d*) .*", lines[feats_anls[page_id]['Коэффициент атерогенности']])[0]
        elif lines[23].startswith('Холестерин общий'):
            page_id = 2
            df_anls.at[sample_id, 'Холестерин общий, ммоль/л'] = re.findall(r"Холестерин общий (\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин общий, ммоль/л']])[0]
            df_anls.at[sample_id, 'Холестерин липопротеидов высокой плотности (ЛПВП, HDL), ммоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин липопротеидов высокой плотности (ЛПВП, HDL), ммоль/л']])[0]
            df_anls.at[sample_id, 'Холестерин не-ЛПВП, ммоль/л'] = re.findall(r"Холестерин не-ЛПВП (\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин не-ЛПВП, ммоль/л']])[0]
            df_anls.at[sample_id, 'Холестерин липопротеидов низкой плотности (ЛПНП, LDL), ммоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Холестерин липопротеидов низкой плотности (ЛПНП, LDL), ммоль/л']])[0]
            df_anls.at[sample_id, 'Гомоцистеин, мкмоль/л'] = re.findall(r"Гомоцистеин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гомоцистеин, мкмоль/л']])[0]
        elif lines[23].startswith('Медь'):
            page_id = 3
            df_anls.at[sample_id, 'Медь, мкмоль/л'] = re.findall(r"Медь (\d+\.*\d*) .*", lines[feats_anls[page_id]['Медь, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Магний, ммоль/л'] = re.findall(r"Магний (\d+\.*\d*) .*", lines[feats_anls[page_id]['Магний, ммоль/л']])[0]
            df_anls.at[sample_id, 'Цинк, мкмоль/л'] = re.findall(r"Цинк (\d+\.*\d*) .*", lines[feats_anls[page_id]['Цинк, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Железо, мкмоль/л'] = re.findall(r"Железо (\d+\.*\d*) .*", lines[feats_anls[page_id]['Железо, мкмоль/л']])[0]
            df_anls.at[sample_id, 'Ферритин, мкг/л'] = re.findall(r"Ферритин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Ферритин, мкг/л']])[0]
        elif lines[24].startswith('ГОРМОНЫ'):
            page_id = 4
            df_anls.at[sample_id, 'Тиреотропный гормон (ТТГ), мкМЕ/мл'] = re.findall(r"Тиреотропный гормон \(ТТГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тиреотропный гормон (ТТГ), мкМЕ/мл']])[0]
            df_anls.at[sample_id, 'Тироксин свободный (Т4 свободный), пмоль/л'] = re.findall(r"Тироксин свободный \(Т4 свободный\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тироксин свободный (Т4 свободный), пмоль/л']])[0]
            df_anls.at[sample_id, 'Трийодтиронин свободный (Т3 свободный), пмоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Трийодтиронин свободный (Т3 свободный), пмоль/л']])[0]
            df_anls.at[sample_id, 'Антитела к тиреоглобулину (Анти-ТГ), МЕ/мл'] = re.findall(r"Антитела к тиреоглобулину \(Анти-ТГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Антитела к тиреоглобулину (Анти-ТГ), МЕ/мл']])[0]
            df_anls.at[sample_id, 'Антитела к микросомальной тиреопероксидазе (Анти-ТПО), МЕ/мл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Антитела к микросомальной тиреопероксидазе (Анти-ТПО), МЕ/мл']])[0]
            df_anls.at[sample_id, 'Фолликулостимулирующий гормон (ФСГ), мМЕ/мл'] = re.findall(r"Фолликулостимулирующий гормон \(ФСГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Фолликулостимулирующий гормон (ФСГ), мМЕ/мл']])[0]
        elif lines[23].startswith('Лютеинизирующий гормон (ЛГ)'):
            page_id = 5
            df_anls.at[sample_id, 'Лютеинизирующий гормон (ЛГ), мМЕ/мл'] = re.findall(r"Лютеинизирующий гормон \(ЛГ\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лютеинизирующий гормон (ЛГ), мМЕ/мл']])[0]
            df_anls.at[sample_id, 'Пролактин, мМЕ/л'] = re.findall(r"Пролактин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Пролактин, мМЕ/л']])[0]
            df_anls.at[sample_id, 'Прогестерон, нг/мл'] = re.findall(r"Прогестерон (\<*\d+\.*\d*) .*", lines[feats_anls[page_id]['Прогестерон, нг/мл']])[0]
        elif lines[23].startswith('Эстрадиол (Е2)'):
            page_id = 6
            df_anls.at[sample_id, 'Эстрадиол (Е2), пмоль/л'] = re.findall(r"Эстрадиол \(Е2\) (\<*\d+\.*\d*) .*", lines[feats_anls[page_id]['Эстрадиол (Е2), пмоль/л']].replace('\u2009', ' '))[0]
            df_anls.at[sample_id, 'Тестостерон свободный, нмоль/л'] = re.findall(r"Тестостерон свободный (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тестостерон свободный, нмоль/л']])[0]
            df_anls.at[sample_id, 'Индекс свободных андрогенов, %'] = re.findall(r"Индекс свободных андрогенов (\d+\.*\d*) .*", lines[feats_anls[page_id]['Индекс свободных андрогенов, %']])[0]
            df_anls.at[sample_id, 'Тестостерон общий, нмоль/л'] = re.findall(r"Тестостерон общий (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тестостерон общий, нмоль/л']])[0]
            df_anls.at[sample_id, 'Глобулин, связывающий половые гормоны (ГСПГ, SHBG), нмоль/л'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Глобулин, связывающий половые гормоны (ГСПГ, SHBG), нмоль/л']])[0]
            df_anls.at[sample_id, 'Инсулин, мкМЕ/мл'] = re.findall(r"Инсулин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Инсулин, мкМЕ/мл']])[0]
        elif lines[25].startswith('Кальций ионизированный'):
            page_id = 7
            df_anls.at[sample_id, 'Кальций ионизированный, ммоль/л'] = re.findall(r"Кальций ионизированный (\d+\.*\d*) .*", lines[feats_anls[page_id]['Кальций ионизированный, ммоль/л']])[0]
        elif lines[24].startswith('ВИТАМИНЫ'):
            page_id = 8
            df_anls.at[sample_id, '25-OH витамин D, ИХЛА, суммарный (кальциферол), нг/мл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['25-OH витамин D, ИХЛА, суммарный (кальциферол), нг/мл']])[0]
        elif lines[24].startswith('ГЕМАТОЛОГИЧЕСКИЕ ИССЛЕДОВАНИЯ'):
            page_id = 9
            df_anls.at[sample_id, 'Эритроциты, 10*12/л'] = re.findall(r"Эритроциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Эритроциты, 10*12/л']])[0]
            df_anls.at[sample_id, 'Гемоглобин, г/л'] = re.findall(r"Гемоглобин (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гемоглобин, г/л']])[0]
            df_anls.at[sample_id, 'Гематокрит, %'] = re.findall(r"Гематокрит (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гематокрит, %']])[0]
            df_anls.at[sample_id, 'Средний объем эритроцитов (MCV), фл'] = re.findall(r"Средний объем эритроцитов \(MCV\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Средний объем эритроцитов (MCV), фл']])[0]
            df_anls.at[sample_id, 'Среднее содержание гемоглобина в эритроците (МСН), пг/кл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Среднее содержание гемоглобина в эритроците (МСН), пг/кл']])[0]
            df_anls.at[sample_id, 'Средняя концентрация Hb в эритроцитах (МСНС), г/дл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Средняя концентрация Hb в эритроцитах (МСНС), г/дл']])[0]
            df_anls.at[sample_id, 'Отн.ширина распред.эритр.по объему (ст.отклонение), фл'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Отн.ширина распред.эритр.по объему (ст.отклонение), фл']])[0]
            df_anls.at[sample_id, 'Отн.ширина распред.эритр.по объему(коэфф.вариации), %'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Отн.ширина распред.эритр.по объему(коэфф.вариации), %']])[0]
            df_anls.at[sample_id, 'Тромбоциты, 10*9/л'] = re.findall(r"Тромбоциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тромбоциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Средний объем тромбоцитов (MPV), фл'] = re.findall(r"Средний объем тромбоцитов \(MPV\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Средний объем тромбоцитов (MPV), фл']])[0]
            df_anls.at[sample_id, 'Тромбокрит (PCT), %'] = re.findall(r"Тромбокрит \(PCT\) (\d+\.*\d*) .*", lines[feats_anls[page_id]['Тромбокрит (PCT), %']])[0]
            df_anls.at[sample_id, 'Относит.ширина распред.тромбоцитов по объему (PDW), %'] = re.findall(r"(\d+\.*\d*) .*", lines[feats_anls[page_id]['Относит.ширина распред.тромбоцитов по объему (PDW), %']])[0]
            df_anls.at[sample_id, 'Лейкоциты, 10*9/л'] = re.findall(r"Лейкоциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лейкоциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Нейтрофилы, 10*9/л'] = re.findall(r"Нейтрофилы (\d+\.*\d*) .*", lines[feats_anls[page_id]['Нейтрофилы, 10*9/л']])[0]
            df_anls.at[sample_id, 'Нейтрофилы, %'] = re.findall(r"Нейтрофилы \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Нейтрофилы, %']])[0]
            df_anls.at[sample_id, 'Эозинофилы, 10*9/л'] = re.findall(r"Эозинофилы (\d+\.*\d*) .*", lines[feats_anls[page_id]['Эозинофилы, 10*9/л']])[0]
            df_anls.at[sample_id, 'Эозинофилы, %'] = re.findall(r"Эозинофилы \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Эозинофилы, %']])[0]
            df_anls.at[sample_id, 'Базофилы, 10*9/л'] = re.findall(r"Базофилы (\d+\.*\d*) .*", lines[feats_anls[page_id]['Базофилы, 10*9/л']])[0]
            df_anls.at[sample_id, 'Базофилы, %'] = re.findall(r"Базофилы \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Базофилы, %']])[0]
            df_anls.at[sample_id, 'Моноциты, 10*9/л'] = re.findall(r"Моноциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Моноциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Моноциты, %'] = re.findall(r"Моноциты \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Моноциты, %']])[0]
            df_anls.at[sample_id, 'Лимфоциты, 10*9/л'] = re.findall(r"Лимфоциты (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лимфоциты, 10*9/л']])[0]
            df_anls.at[sample_id, 'Лимфоциты, %'] = re.findall(r"Лимфоциты \% (\d+\.*\d*) .*", lines[feats_anls[page_id]['Лимфоциты, %']])[0]
        elif lines[25].startswith('Гликированный гемоглобин А1с'):
            page_id = 10
            df_anls.at[sample_id, 'Гликированный гемоглобин А1с, %'] = re.findall(r"Гликированный гемоглобин А1с (\d+\.*\d*) .*", lines[feats_anls[page_id]['Гликированный гемоглобин А1с, %']])[0]
    
df_anls = df_anls.apply(pd.to_numeric, errors='ignore')


# Electrocardiogram

In [ ]:
feats_rows_ecg = {
    'ID': 11,
    'Дата и время обследования': 22,
    'Возраст': 14,
    'Пол': 13,
    'Длительность P, мс': 32,
    'Длительность QRS, мс': 41,
    'Интервал PQ, мс': 31,
    'Интервал QT, мс': 40,
    'Интервал QTc, мс': 47,
    'Ось P': 30,
    'Ось QRS': 39,
    'Ось T': 46,
    'ЧСС, уд/мин': 25,
    'Нерегулярность ритма, %': 51
}

fns = [
    'Гуржий Юлия Андреевна 1982',
    'Задоя Ирина Петровна',
    'Бурминская Светлана Александровна 2024',
    'Вотина Полина Игоревна',
    'Мустафина Диана',
    'Нестерова Наталья Григорьена'
]

df_ecg = pd.DataFrame(columns=list(feats_rows_ecg.keys()))

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")
    
    # Page 1
    page = reader.pages[0]
    lines = page.extract_text().splitlines()
    if lines[16] == 'Дата/время:':
        row_id_inc = -1
    else:
        row_id_inc = 0
    sample_id = lines[feats_rows_ecg['ID']]
    df_ecg.at[sample_id, 'ID'] = sample_id
    df_ecg.at[sample_id, 'Дата и время обследования'] = lines[feats_rows_ecg['Дата и время обследования'] + row_id_inc]
    df_ecg.at[sample_id, 'Возраст'] = float(re.findall(r"(\d+) .*", lines[feats_rows_ecg['Возраст']])[0])
    df_ecg.at[sample_id, 'Пол'] = lines[feats_rows_ecg['Пол']][0].upper()
    df_ecg.at[sample_id, 'Длительность P, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Длительность P, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Длительность QRS, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Длительность QRS, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Интервал PQ, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Интервал PQ, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Интервал QT, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Интервал QT, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Интервал QTc, мс'] = re.findall(r"\s*(\d+)\s*мс", lines[feats_rows_ecg['Интервал QTc, мс'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Ось P'] = re.findall(r"\s*(-?[0-9]\d*(\.\d+)?)", lines[feats_rows_ecg['Ось P'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Ось QRS'] = re.findall(r"\s*(-?[0-9]\d*)", lines[feats_rows_ecg['Ось QRS'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Ось T'] = re.findall(r"\s*(-?[0-9]\d*)", lines[feats_rows_ecg['Ось T'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'ЧСС, уд/мин'] = re.findall(r"\s*(\d+)\s*уд\/мин", lines[feats_rows_ecg['ЧСС, уд/мин'] + row_id_inc])[0]
    df_ecg.at[sample_id, 'Нерегулярность ритма, %'] = re.findall(r"\s+Нерегулярность\s+ритма\s+=\s+(\d+)\%", lines[feats_rows_ecg['Нерегулярность ритма, %'] + row_id_inc])[0]
    
df_ecg = df_ecg.apply(pd.to_numeric, errors='ignore')

# Different analyses 2

In [ ]:
fns = [
    '7.2.A5.201 - Прогестерон, 7.2.A4.201 - Эстрадиол (Е2), 50.0.H57.201 - Тестостерон свободный (включает определение тестостерона общего и свободного, Г',
    '50.0.H198 - Чекап Елена Яровова, 4.7.A8.201 - Медь',
    '1.0.A1.202 - СОЭ (венозная кровь), 4.2.D1.201 - Белковые фракции (включает определение общего белка и альбумина), 4.8.A1.201 - Железо , 1.0.D2.202 - Клиниче',
    '1.0.D2.202 - Клинический анализ крови с лейкоцитарной формулой (5DIFF) (венозная кровь), 4.2.A2.201 - Общий белок',
    '4.1.A2.201 - Аспартатаминотрансфераза (АСТ) , 4.5.A10.201 - Гомоцистеин, 4.2.A2.201 - Общий белок, 4.7.A8.201 - Медь, 4.1.A5.201 - Гамма-глутамилтрансфераза (ГГТ',
    '4.2.A4.201 - Мочевина, 7.1.A3.201 - Трийодтиронин свободный (Т3 свободный), 7.1.A2.201 - Тироксин свободный (Т4 свободный), 4.2.A5.201 - Мочевая кислота, 4.2.A'
]

feats = pd.read_excel(f"{path}/feats.xlsx", index_col=0)
feats_dict = dict(zip(feats.index.values, feats['feature'].values))

df_anls = pd.DataFrame(columns=['ID', 'Возраст', 'Пол', 'Дата и время'] + list(feats['feature']))
missed_lines = []

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")

    lines = []
    for page in reader.pages:
        lines += page.extract_text().splitlines()

    sample_info = re.findall(r"Пациент: (.+), (\d+) .+, (.+)", lines[3])[0]
    date_and_time = re.findall(r"Дата приёма: (.*)", lines[1])[0]
    sample_id = f"{sample_info[0]} {date_and_time}"
    df_anls.at[sample_id, 'ID'] = sample_info[0]
    print(sample_id)
    df_anls.at[sample_id, 'Возраст'] = sample_info[1]
    df_anls.at[sample_id, 'Пол'] = sample_info[2][0].upper()
    df_anls.at[sample_id, 'Дата и время'] = date_and_time

    for line in lines[7::]:
        line_parse = re.findall(fr"(.*): ([-+]?(?:\d+\.\d+|\d+|\.\d+))\s*", line)
        if len(line_parse) > 0:
            if line_parse[0][0] in feats_dict:
                df_anls.at[sample_id, feats_dict[line_parse[0][0]]] = line_parse[0][1]
            else:
                missed_lines.append(line)
        else:
            missed_lines.append(line)

df_anls = df_anls.apply(pd.to_numeric, errors='ignore')

In [ ]:
line = lines[31]
line_parse = re.findall(fr"(.*): ([-+]?(?:\d+\.\d+|\d+|\.\d+))\s*", line)[0]

In [ ]:
# extract only text oriented up
print(page.extract_text(0))

# extract text oriented up and turned left
print(page.extract_text((0, 90)))

# extract text in a fixed width format that closely adheres to the rendered
# layout in the source pdf
print(page.extract_text(extraction_mode="layout"))

# extract text preserving horizontal positioning without excess vertical
# whitespace (removes blank and "whitespace only" lines)
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

# adjust horizontal spacing
print(page.extract_text(extraction_mode="layout", layout_mode_scale_weight=1.0))

# exclude (default) or include (as shown below) text rotated w.r.t. the page
print(page.extract_text(extraction_mode="layout", layout_mode_strip_rotated=False))

In [ ]:
reader = PdfReader(f"{path}/Бурминская Светлана Александровна, #2, 2024-10-11 10_55_46.pdf")
page = reader.pages[0]
print(page.extract_text(0))

In [ ]:
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

In [ ]:
import re

def extract_floats(text):
    # Паттерн для поиска чисел с плавающей точкой в указанном формате
    pattern = r":\s*(\d+\.\d+)\s*кг"
    # Находим все совпадения в тексте
    matches = re.findall(pattern, text)
    # Конвертируем найденные строки в числа типа float
    return [float(match) for match in matches]

# Пример использования
text = """
Ваша активная клеточная масса составляет: 24.1 кг (диапазон)
Другие показатели: жировая масса 15.5 кг, вода: 30.0 кг.
Еще один пример: 42.8 кг (невероятный результат)!
"""

floats = extract_floats(text)
print("Найденные числа:", floats)

In [ ]:
import re

text = "39, ЖВозраст, лет / Пол"
result = re.findall(r"(\d+), (\S)Возраст.*", text)